In [1]:
# import libraries
import pandas as pd
import random

In [2]:
# define a function to create a 0-back block
def create_0back_block(target, n_stimuli, stimuli, target_percentage):
    stimuli_list = []
    trial_type_list = []
    stimuli_onset_list = []
    n_target = int(n_stimuli * target_percentage)
    n_non_target = n_stimuli - n_target
    onset_counter = 0.0

    # Create target and non-target lists
    target_list = ['target'] * n_target
    non_target_list = ['non-target'] * n_non_target

    # Combine and shuffle the lists
    trial_type = target_list + non_target_list
    random.shuffle(trial_type)

    # if trial_type is target, then the stimulus is 'X'
    for trial in trial_type:
        if trial == 'target':
            stimuli_list.append(target)
            trial_type_list.append(trial)
            stimuli_onset_list.append(onset_counter)
            onset_counter += 2.8
        # if trial_type is non-target, then the stimulus is a random letter in stimuli letter list
        elif trial == 'non-target':
            stimuli_list.append(random.choice(stimuli))
            trial_type_list.append(trial)
            stimuli_onset_list.append(onset_counter)
            onset_counter += 2.8
        
        # Add fixation stimulus
        stimuli_list.append('+')
        trial_type_list.append('fixation')
        stimuli_onset_list.append(onset_counter)
        onset_counter += 0.2

    # Remove the last fixation stimulus
    stimuli_list = stimuli_list[:-1]
    trial_type_list = trial_type_list[:-1]
    stimuli_onset_list = stimuli_onset_list[:-1]
    
    # create a dataframe with the stimuli and trial_type
    df = pd.DataFrame({'stimuli': stimuli_list, 'trial_type': trial_type_list, "stimuli_onset" : stimuli_onset_list})
    
    # Add block_type column
    df['block_type'] = '0-back'

    return df

In [3]:
# define a function to create 1-back block
def create_1back_block(n_stimuli, stimuli, target_percentage):
    stimuli_list = []
    trial_type_list = []
    stimuli_onset_list = []
    n_target = int(n_stimuli * target_percentage)
    n_non_target = n_stimuli - n_target
    onset_counter = 0.0

    # Create target and non-target lists
    target_list = ['target'] * n_target
    non_target_list = ['non-target'] * (n_non_target - 1)  # Subtract 1 for the initial non-target

    # Combine and shuffle the lists
    trial_type = target_list + non_target_list
    random.shuffle(trial_type)

    # Start with a non-target stimulus
    last_stimulus = random.choice(stimuli)
    stimuli_list.append(last_stimulus)
    trial_type_list.append('non-target')
    stimuli_onset_list.append(onset_counter)
    onset_counter += 2.8

    # Add fixation after the first non-target stimulus
    stimuli_list.append('+')
    trial_type_list.append('fixation')
    stimuli_onset_list.append(onset_counter)
    onset_counter += 0.2

    # if trial_type is target, then the stimulus is the same as the last stimulus
    for trial in trial_type:
        if trial == 'target':
            stimuli_list.append(last_stimulus)
            trial_type_list.append(trial)
            stimuli_onset_list.append(onset_counter)
            onset_counter += 2.8
        # if trial_type is non-target, then the stimulus is a random letter in stimuli letter list
        elif trial == 'non-target':
            # Choose a stimulus that is different from the last stimulus
            last_stimulus = random.choice([l for l in stimuli if l != last_stimulus])
            stimuli_list.append(last_stimulus)
            trial_type_list.append(trial)
            stimuli_onset_list.append(onset_counter)
            onset_counter += 2.8
        
        # Add fixation stimulus
        stimuli_list.append('+')
        trial_type_list.append('fixation')
        stimuli_onset_list.append(onset_counter)
        onset_counter += 0.2

    # Remove the last fixation stimulus
    stimuli_list = stimuli_list[:-1]
    trial_type_list = trial_type_list[:-1]
    stimuli_onset_list = stimuli_onset_list[:-1]

    # create a dataframe with the stimuli and trial_type
    df = pd.DataFrame({'stimuli': stimuli_list, 'trial_type': trial_type_list, 'stimuli_onset': stimuli_onset_list})
    
    # Add block_type column
    df['block_type'] = '1-back'

    return df

In [6]:
# define a function to create the experiment
def create_experiment(n_blocks, target, n_stimuli, stimuli, target_percentage):
    blocks = []
    for i in range(n_blocks):
        # Add a cue at the beginning of each block
        cue_df = pd.DataFrame({'stimuli': ['png'], 'trial_type': ['cue'], 'stimuli_onset': [0.0], 'block_type': ['cue']})
        blocks.append(cue_df)

        # Alternate between 0-back and 1-back blocks
        if i % 2 == 0:
            block = create_0back_block(target, n_stimuli, stimuli, target_percentage)
            block['block_type'] = '0-back'
        else:
            block = create_1back_block(n_stimuli, stimuli, target_percentage)
            block['block_type'] = '1-back'
        blocks.append(block)

        # Add a rest period at the end of each block
        rest_df = pd.DataFrame({'stimuli': ['+'], 'trial_type': ['rest'], 'stimuli_onset': [0.0], 'block_type': ['rest']})
        blocks.append(rest_df)

    # Concatenate all blocks into a single DataFrame
    experiment = pd.concat(blocks).reset_index(drop=True)

    # Adjust the stimuli_onset times
    onset_counter = 0.0
    for i, row in experiment.iterrows():
        experiment.at[i, 'stimuli_onset'] = onset_counter
        if row['trial_type'] == 'rest':
            onset_counter += 10.0  # Rest periods last 1 second
        elif row['trial_type'] == 'cue':
            onset_counter += 2.0  # Cues last 2 seconds
        elif row['trial_type'] == 'fixation':
            onset_counter += 0.2
        else:
            onset_counter += 2.8  # Stimuli last 2.8 seconds
    
    # Add duration column
    experiment['stimulus_duration'] = 0.0
    for i, row in experiment.iterrows():
        if row['trial_type'] == 'cue':
            experiment.at[i, 'stimulus_duration'] = 2.0
        elif row['trial_type'] == 'rest':
            experiment.at[i, 'stimulus_duration'] = 10.0
        elif row['trial_type'] == 'fixation':
            experiment.at[i, 'stimulus_duration'] = 0.2
        else:
            experiment.at[i, 'stimulus_duration'] = 2.8   

    # Add cor_res column
    experiment['cor_res'] = experiment['trial_type'].apply(lambda x: 'left' if x == 'target' else 'right' if x == 'non-target' else '')


    return experiment

In [7]:
# create the experiment
exp = create_experiment(n_blocks=8, # total number of 0- and 1-back blocks
                        target='X', # target stimulus for 0-back block
                        n_stimuli=10, # number of stimuli (trial) per block
                        stimuli=['P','Q','R','T','K','Z','H','E','U'], # list of stimuli
                        target_percentage=0.5) # percentage of target stimuli in each block

# save the experiment to a csv file
exp.to_csv('experiment.csv', index=False)

exp

,stimuli,trial_type,stimuli_onset,block_type,stimulus_duration,cor_res
0,png,cue,0.0,cue,2.0,
1,X,target,2.0,0-back,2.8,left
2,+,fixation,4.8,0-back,0.2,
3,P,non-target,5.0,0-back,2.8,right
4,+,fixation,7.8,0-back,0.2,
...,...,...,...,...,...,...
163,+,fixation,318.4,1-back,0.2,
164,E,target,318.6,1-back,2.8,left
165,+,fixation,321.4,1-back,0.2,
166,E,target,321.6,1-back,2.8,left
